# Natural Statistics Cross-linguistic: 

#### Lexical diversity analysis - random sample 

----

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "data_proc")
import analytic_proc
import warnings
warnings.filterwarnings('ignore')

In [2]:
TSE_dat_inc = pd.read_csv("../data/TSE_cont_dat.csv")
TSE_rand_time_chunks = pd.read_csv("../data/TSE_rand_time_chunks.csv")
TSE_dat_inc.rename(columns={'Langauge_name': 'Language_name'}, inplace=True)
TSE_dat_inc['target_child_id'] = TSE_dat_inc['transcript_id']

In [3]:
def is_within_time_chunk(transcript):
    transcript_id, transcript_rows = transcript
    time_chunks = TSE_rand_time_chunks[TSE_rand_time_chunks['transcript_id'] == transcript_id]
    mask = transcript_rows.apply(lambda row: any(time_chunk['media_start'] <= row['media_start'] and time_chunk['media_end'] >= row['media_end'] for _, time_chunk in time_chunks.iterrows()), axis=1)
    return transcript_rows[mask]

In [4]:
TSE_dat_inc_grouped = TSE_dat_inc.groupby('transcript_id')

In [5]:
TSE_rand_dat_inc = pd.concat([is_within_time_chunk(transcript) for transcript in TSE_dat_inc_grouped])

In [6]:
rand_dat_inc = pd.read_csv("../data/rand_dat_inc_master.csv",index_col=0,low_memory=False)

In [7]:
# load and clean data
rand_dat_inc=rand_dat_inc[rand_dat_inc["language"]!="ara"]
rand_dat_inc=rand_dat_inc[(rand_dat_inc["target_child_age"]>=5) & (rand_dat_inc["target_child_age"]<=30)]

In [8]:
# this way we include older and younger Tseltal kids
rand_dat_inc = pd.concat([TSE_rand_dat_inc, rand_dat_inc])

In [9]:
rand_dat_inc_cg = rand_dat_inc[rand_dat_inc["caregiver"]=="caregiver"]

rand_dat_inc_cg["contingent"] = np.where(rand_dat_inc_cg["contingent"]==1, "contingent", "non-contingent")

rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"].notna()]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="xxx"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="yyy"]
rand_dat_inc_cg = rand_dat_inc_cg[rand_dat_inc_cg["gloss"]!="www"]

In [10]:
TSE_dat_test = rand_dat_inc[rand_dat_inc["Language_name"]=="Tseltal"]

Would call the following if we could run lexical diversity analysis globally:

```python
analytic_proc.create_result(rand_dat_inc_cg)
```

However, we want to have seperate dictionaries for contingent and non-contingent words so we can compare them to one another.

The function will allow us to have a different dictionary for each transcript.

Finally, to compare, we can run mixed effects to understand whether contingent and non-contingent utterances differ in their lexical diversity.

----
#### Seperate contingent and non-contingent utterances into individual dataframes

In [11]:
rand_dat_inc_cg_cc = rand_dat_inc_cg[rand_dat_inc_cg["contingent"]=="contingent"].reset_index(drop=True)
rand_dat_inc_cg_nc = rand_dat_inc_cg[rand_dat_inc_cg["contingent"]=="non-contingent"].reset_index(drop=True)

----
#### Loop through each unique transcript to compute the lexical diversity counts across languages.

In [12]:
analytic_proc.create_c_result(rand_dat_inc_cg_cc,"rand")

In [13]:
analytic_proc.create_nc_result(rand_dat_inc_cg_nc,"rand")

----
#### Lexical Diversity plot

In [14]:
rand_dat_inc_cg_cc = pd.read_csv("../data/rand_dat_inc_master_cc_lexdiv.csv",index_col=0,low_memory=False)
rand_dat_inc_cg_nc = pd.read_csv("../data/rand_dat_inc_master_nc_lexdiv.csv",index_col=0,low_memory=False)

# combine dataframes into one

rand_dat_inc_cg = pd.concat([rand_dat_inc_cg_cc,rand_dat_inc_cg_nc])

In [15]:
# add play context and year of study

play_context = pd.read_csv("../data/context_data.csv")
play_context = play_context.rename(columns={"Corpus": "corpus_name"})

# print(play_context.to_markdown())

rand_dat_inc_cg = rand_dat_inc_cg.merge(play_context,on='corpus_name')

rand_dat_inc_cg["context"] = rand_dat_inc_cg["Location"] + rand_dat_inc_cg["Activity"]

rand_dat_inc_cg["context"] = rand_dat_inc_cg["context"].replace({"HomeBook-reading":"Home: book reading",
                                                                 "HomeInterview/Unstructured":"Home: interview/unstructured",
                                                                 "HomeNaN":"Home: unreported",
                                                                 "HomeOther":"Home: other",
                                                                 "HomeUnstructured":"Home: unstructured",
                                                                 "LabOther":"Lab: other",
                                                                 "LabTabletop play":"Lab: tabletop play",
                                                                 "LabInterview/Unstructured":"Lab: interview/unstructured",
                                                                 "LabUnstructured":"Lab: unstructured",
                                                                 np.nan:"Unreported",
                                                                 "OtherUnstructured":"Other: unstructured"})

# year of study
# corpora_year = pd.read_csv("../data/corpora_year.csv")
# corpora_year = corpora_year.rename(columns={"Corpora": "corpus_name"})
# corpora_year = corpora_year[["corpus_name", "Year collected"]]

# rand_dat_inc_cg = rand_dat_inc_cg.merge(corpora_year,on='corpus_name')

In [16]:
len(rand_dat_inc_cg["Language_name"].unique())

15

In [17]:
# age descriptives
ar = [['min', rand_dat_inc_cg["target_child_age"].min()],
      ['max', rand_dat_inc_cg["target_child_age"].max()],
      ['mean', rand_dat_inc_cg["target_child_age"].mean()],
      ['stdev', rand_dat_inc_cg["target_child_age"].std()]]

age_range = pd.DataFrame(ar, columns = ['stat', 'age'])

# inspect data:
print(age_range.to_markdown())

|    | stat   |      age |
|---:|:-------|---------:|
|  0 | min    |  2       |
|  1 | max    | 36       |
|  2 | mean   | 19.1692  |
|  3 | stdev  |  6.73674 |


In [18]:
%load_ext rpy2.ipython

In [19]:
%%R -i rand_dat_inc_cg

library("lme4")
library("repr")
library("knitr")
library("broom")
library("emmeans")
library("tidyverse")
library("kableExtra")

Loading required package: Matrix
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()

Attaching package: ‘kableExtra’

The following object is masked from ‘package:dplyr’:

    group_rows



In [20]:
%%R -o rand_dat_inc_cg

# ---- create caregiver type categories

caregiver_type <- rand_dat_inc_cg %>%
  group_by(transcript_id) %>%
  summarise(
    caregiver_type = case_when(
      all(speaker_role == "Mother") ~ "Mother only",
      all(speaker_role == "Father") ~ "Father only",
      any(speaker_role %in% c("Mother", "Father")) ~ "Mother & Father",
      TRUE ~ "Unknown"
    )
  )

# inspect data:
# caregiver_type %>%
#     kbl(format="pipe")
    
# ggplot(caregiver_type, aes(x = 1, y = caregiver_type, fill = factor(caregiver_type))) + 
#   geom_col() +
#   coord_polar(theta = "y") +
#   theme_void()

rand_dat_inc_cg <- rand_dat_inc_cg %>%
  left_join(caregiver_type)

Joining with `by = join_by(transcript_id)`


In [21]:
rand_lex_stats = (rand_dat_inc_cg.groupby(["Language_name","target_child_id","transcript_id","contingent"]) #,"context","Year collected"])
                                  .uniquenss
                                  .agg(["sum"])
                                  .reset_index())
rand_lex_sumstats = rand_lex_stats.rename({'sum': 'sums'}, axis=1)

In [22]:
%%R -i rand_lex_sumstats

# ^import rand_lex_sumstats into R

NULL


In [23]:
%%R -o rand_lex_sumstats

rand_lex_sumstats <- rand_lex_sumstats %>%
    left_join(caregiver_type) %>%
    filter(Language_name != "Mandarin") %>%
    filter(Language_name != "Polish")

Joining with `by = join_by(transcript_id)`


In [24]:
# inspect data:
len(rand_lex_sumstats["Language_name"].unique())

13

In [25]:
# inspect data:
len(rand_lex_sumstats["transcript_id"].unique())

1585

In [26]:
# save data to file:
rand_lex_sumstats.to_csv("../data/rand_lex_sumstats.csv")

Simple plot

In [27]:
%%R -i rand_lex_sumstats

options(repr.plot.width=6, repr.plot.height=12, scipen=999)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="English")
est_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Norwegian")
pol_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(sums=c(240),contingent = c(1.5),Language_name="Portuguese")
spa_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Swedish")
tse_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Tseltal")
# zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_lex_sumstats, aes(x = contingent, y = sums, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .5) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=8,color="black") + 
     geom_text(data = eng_label,label = "***",size=8,color="black") +  
     geom_text(data = est_label,label = "**",size=8,color="black") +  
#      geom_text(data = fas_ns_label,label = "*",size=4,color="black",fontface = "italic") +
     geom_text(data = fas_label,label = "*",size=8, color="black") +
     geom_text(data = fra_label,label = "***",size=8,color="black") +  
     geom_text(data = hrv_label,label = "***",size=8,color="black") + 
     geom_text(data = jpn_label,label = "***",size=8,color="black") + 
     geom_text(data = kor_label,label = "***",size=8,color="black") +  
     geom_text(data = nor_ns_label,label = "ns",size=4,color="black",fontface = "italic") +  
#      geom_text(data = pol_ns_label,label = "ns", size=4,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "**",size=8,color="black") +  
     geom_text(data = spa_ns_label,label = "ns",size=4,color="black",fontface = "italic") + 
     geom_text(data = swe_label,label = "ns",size=4,color="black",fontface = "italic") + 
     geom_text(data = tse_label,label = "", size=8, color="black") +
#      geom_text(data = zho_ns_label,label = "ns",size=4,color="black",fontface = "italic") +
     # geom_text(data = zho_label,label = "^",size=8, color="black") +
     ylim(0, 250) +
     labs(tag = "A",
          y = "Number of Unique Words", x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=16),
           axis.text.x = element_text(vjust = 0.5, hjust=.5),
           legend.title = element_blank(),
           legend.background = element_rect(fill=alpha("white",0.90),
                                                            size=0, linetype="dotted",
                                                            colour = "white"),
           legend.text=element_text(size=16))
     ggsave("../figures/lexical_diversity_rand.pdf", width = 11.7, height = 6.2)

In addition: Warning messages:
1: The `fun.y` argument of `stat_summary()` is deprecated as of ggplot2 3.3.0.
ℹ Please use the `fun` argument instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
2: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
3: The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
4: Removed 60 rows containing non-finite values (`stat_summary()`). 
5: Removed 60 rows containing non-finite values (`stat_summary()`). 


for manuscript

In [28]:
%%R -i rand_lex_sumstats

library('ggplot2')
library('repr')
options(repr.plot.width=6, repr.plot.height=12)

xlabs <- c("C", "NC")

# ara_label <- data.frame(means=c(0),contingent = c(1.5),language="ara") # no adult speech transcribed
deu_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="German")
eng_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="English")
est_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Estonian")
fas_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Persian")
fra_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="French")
hrv_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Croatian")
jpn_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Japanese")
kor_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Korean")
# nor_label <- data.frame(means=c(5.8),contingent = c(1.5),Language_name="Norwegian")
nor_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Norwegian")
# pol_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Polish")
por_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Portuguese")
spa_ns_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Spanish")
swe_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Swedish")
tse_label <- data.frame(sums=c(247),contingent = c(1.5),Language_name="Tseltal")
# zho_label <- data.frame(sums=c(235),contingent = c(1.5),Language_name="Mandarin")
# zho_ns_label <- data.frame(means=c(6),contingent = c(1.5),Language_name="Mandarin")


p <- ggplot(rand_lex_sumstats, aes(x = contingent, y = sums, color = Language_name)) +
     stat_summary(fun.y=mean, geom="point", shape=19, size=1.75) + 
     stat_summary(fun.data = mean_se, geom = "errorbar", size=1.25, width = .4) +
     facet_wrap(. ~ Language_name,ncol = 7) + 
     geom_text(data = deu_label,label = "***",size=6,color="black") + 
     geom_text(data = eng_label,label = "***",size=6,color="black") +  
     geom_text(data = est_label,label = "**",size=6,color="black") +  
     geom_text(data = fas_label,label = "*",size=6, color="black") +
     geom_text(data = fra_label,label = "***",size=6,color="black") +  
     geom_text(data = hrv_label,label = "***",size=6,color="black") + 
     geom_text(data = jpn_label,label = "***",size=6,color="black") + 
     geom_text(data = kor_label,label = "***",size=6,color="black") +  
     geom_text(data = nor_ns_label,label = "ns",size=3,color="black",fontface = "italic") +  
     geom_text(data = por_label,label = "**",size=6,color="black") +  
     geom_text(data = spa_ns_label,label = "ns",size=3,color="black",fontface = "italic") + 
     geom_text(data = swe_label,label = "ns",size=3,color="black",fontface = "italic") + 
     geom_text(data = tse_label,label = "",size=6,color="black") + 
     # geom_text(data = zho_label,label = "^",size=3, color="black") +
     ylim(0, 250) +
     labs(tag = "A",
          y = "Number of Unique Words", x = "") +
     theme_classic() +
     scale_x_discrete(labels= xlabs) +
     theme(text = element_text(size=11.5),
           axis.text.x = element_text(vjust = 0.5, hjust=0.5),
           legend.position="none")
     ggsave("../figures/figure_2_A.pdf", width = 11.5, height = 4.2)

In addition: Warning messages:
1: Removed 60 rows containing non-finite values (`stat_summary()`). 
2: Removed 60 rows containing non-finite values (`stat_summary()`). 


----
#### Statistical analyses

By language

In [29]:
rand_lex_sumstats = rand_lex_sumstats[['Language_name','sums','contingent','transcript_id','target_child_id','caregiver_type']] #, 'Year collected']]

rand_dat_inc_cg_count = rand_dat_inc_cg[['Language_name','transcript_id','target_child_id']]

In [30]:
rand_dat_inc_cg_count.to_csv("../data/rand_dat_inc_cg_count.csv")

In [31]:
%%R -i rand_lex_sumstats -i rand_dat_inc_cg_count

# vectors for rows to remove from lmer
case_study <- c("Persian") # only 1 target child analyzed

case_study_cgtype_compare <- c("Korean") # only 1 target child analyzed, varies in CG type

no_cgtype_compare <- c("Portuguese", "Tseltal") # only `Mother only`

# single_tran <- c("Polish") # only 1 transcript

# nests of models
lexdiv_nest1 <- rand_lex_sumstats %>%
    filter(!Language_name %in% case_study) %>%
    filter(!Language_name %in% no_cgtype_compare) %>%
    filter(!Language_name %in% case_study_cgtype_compare) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(sums ~ contingent + caregiver_type +
                                (1|target_child_id) +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

lexdiv_nest2 <- rand_lex_sumstats %>%
    filter(Language_name %in% case_study_cgtype_compare) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(sums ~ contingent + caregiver_type +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

lexdiv_nest3 <- rand_lex_sumstats %>%
    filter(Language_name %in% no_cgtype_compare) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(sums ~ contingent + 
                                (1|target_child_id) +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

lexdiv_nest4 <- rand_lex_sumstats %>%
    filter(Language_name %in% case_study) %>%
    group_by(Language_name) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(sums ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(Language_name, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))
    
# number of transcripts per language
sample_size <- rand_dat_inc_cg_count %>%
    group_by(Language_name) %>%
    summarize(n = n_distinct(transcript_id))
    
# combine lmer summaries and correct p-values for multiple comparisons
emms_all <- list(lexdiv_nest1, lexdiv_nest2, lexdiv_nest3, lexdiv_nest4) %>% 
    reduce(bind_rows) %>%
    mutate(p.value = p.adjust(p.value, "holm", 13)) %>%
    left_join(sample_size)

Joining with `by = join_by(Language_name)`


boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
In addition: Warning messages:
1: `cols` is now required when using `unnest()`.
ℹ Please use `cols = c(effect_size)`. 
2: `cols` is now required when using `unnest()`.
ℹ Please use `cols = c(effect_size)`. 
3: There were 4 warnings in `mutate()`.
The first warning was:
ℹ In argument: `fit = map(...)`.
ℹ In group 2: `Language_name = "Tseltal"`.
Caused by warning in `checkConv()`:
! unable to evaluate scaled gradient
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 rem

format statistics table

In [32]:
%%R

table_maker = function(data) { data %>%
    select(Language_name, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Language", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,Language)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Language (n)", c('Language','n'), sep=" (") %>%
    mutate(`Language (n)` = paste0(`Language (n)`,")")) %>%
    arrange(`Language (n)`)
    }

lexdiv_stats_table <- table_maker(emms_all)

kable(lexdiv_stats_table,"pipe")



|Language (n)    |Estimate (SE)  | Test statistic| Effect size|Adjusted p-value |
|:---------------|:--------------|--------------:|-----------:|:----------------|
|Croatian (58)   |-37.14 (3.93)  |          -9.44|       -1.77|<.0001           |
|English (882)   |-105.61 (2.04) |         -51.73|       -2.49|<.0001           |
|Estonian (22)   |-69.36 (15.44) |          -4.49|       -1.40|0.0019           |
|French (279)    |-68.39 (3.3)   |         -20.71|       -1.76|<.0001           |
|German (38)     |-66.55 (7.93)  |          -8.39|       -1.95|<.0001           |
|Japanese (160)  |-51.52 (3.38)  |         -15.24|       -1.71|<.0001           |
|Korean (28)     |-107 (12.68)   |          -8.44|       -2.32|<.0001           |
|Norwegian (28)  |-32.1 (12.31)  |          -2.61|       -0.72|0.1872           |
|Persian (11)    |-63.36 (12.9)  |          -4.91|       -2.20|0.0046           |
|Portuguese (23) |-39.39 (9.32)  |          -4.23|       -1.27|0.0038           |
|Spanish (30) 

In [30]:
%%R 

# add columns sample and measure and save

lexdiv_stats_table %>%
    mutate(sample = "rand",
           measure = "lexdiv") %>%
    write.csv(file = "../data/rand_lexdiv_stats.csv")

In [31]:
%%R 

# ---- Sign test per language

rand_lex_stats_wide <- rand_lex_sumstats %>%
  spread(key = contingent, value = sums) %>%
  mutate(diff = `non-contingent` - contingent,
         sign_cat = case_when(
           diff > 0 ~ "+",
           diff < 0 ~ "-",
           TRUE ~ "0")) %>%
  group_by(Language_name) %>%
  count(sign_cat) %>%
  mutate(prop = n / sum(n)) %>%
  filter(sign_cat == "+")

rand_lex_stats_wide %>%
  mutate(measure = "Number of Unique Words") %>%
  write.csv(file = "../data/rand_lexdiv_signtest.csv")

kbl(rand_lex_stats_wide,"pipe")



|Language_name |sign_cat |   n|      prop|
|:-------------|:--------|---:|---------:|
|Croatian      |+        |  53| 0.9137931|
|English       |+        | 839| 0.9512472|
|Estonian      |+        |  18| 0.8181818|
|French        |+        | 245| 0.8781362|
|German        |+        |  36| 0.9473684|
|Japanese      |+        | 145| 0.9062500|
|Korean        |+        |  25| 0.8928571|
|Mandarin      |+        |   2| 1.0000000|
|Norwegian     |+        |  16| 0.5714286|
|Persian       |+        |  11| 1.0000000|
|Polish        |+        |   1| 1.0000000|
|Portuguese    |+        |  20| 0.8695652|
|Spanish       |+        |  15| 0.5000000|
|Swedish       |+        |  13| 0.8125000|


By play context

In [32]:
rand_lex_sumstats_contex = (rand_dat_inc_cg.groupby(["context","target_child_id","transcript_id","contingent"])
                                  .uniquenss
                                  .agg(["sum"])
                                  .reset_index())

rand_lex_sumstats_contex =  rand_lex_sumstats_contex.rename({'sum': 'sums'}, axis=1)

In [33]:
rand_lex_sumstats_contex["context"].unique()

array(['Home: book reading', 'Home: interview/unstructured',
       'Home: other', 'Home: unstructured', 'Lab: interview/unstructured',
       'Lab: unstructured', 'Other: unstructured', 'Unreported'],
      dtype=object)

In [34]:
%%R -i rand_lex_sumstats_contex

# vectors for rows to remove from lmer
single_tran <- c("Home: interview/unstructured") # only 1 transcript

contex_sample_size <- rand_lex_sumstats_contex %>%
    group_by(context) %>%
    summarize(n = n_distinct(transcript_id))

lexdiv_contex_nest_1 <- rand_lex_sumstats_contex %>%
    filter(!context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(sums ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(context, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))
    
lexdiv_contex_nest_2 <- rand_lex_sumstats_contex %>%
    filter(context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lm(sums ~ contingent,
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise")))) %>%
    select(context, contrasts) %>%
    unnest(cols = c(contrasts))  %>%
    rename(statistic = `t.ratio`)

# combine lmer summaries and correct p-values for multiple comparisons
context_emms_all <- list(lexdiv_contex_nest_1, lexdiv_contex_nest_2) %>% 
    reduce(bind_rows) %>%
    mutate(p.value = p.adjust(p.value, "holm", 7)) %>%
    left_join(contex_sample_size)

R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Joining with `by = join_by(context)`


In [36]:
%%R

table_maker = function(data) { data %>%
    select(context, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Play context", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Play context`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Play context (n)", c(`Play context`,'n'), sep=" (") %>%
    mutate(`Play context (n)` = paste0(`Play context (n)`,")")) %>%
    arrange(`Play context (n)`)
    }
    
lexdiv_context_stats_table <- table_maker(context_emms_all)

kable(lexdiv_context_stats_table, "pipe")



|Play context (n)                  |Estimate (SE)  | Test statistic| Effect size|Adjusted p-value |
|:---------------------------------|:--------------|--------------:|-----------:|:----------------|
|Home: book reading (28)           |-107 (12.81)   |          -8.36|       -2.27|<.0001           |
|Home: interview/unstructured (1)  |-7 (NaN)       |            NaN|          NA|NaN              |
|Home: other (20)                  |-46.8 (5.34)   |          -8.77|       -2.84|<.0001           |


|Home: unstructured (902)          |-72.72 (1.93)  |         -37.63|       -1.78|<.0001           |
|Lab: interview/unstructured (368) |-141.79 (2.85) |         -49.76|       -3.74|<.0001           |
|Lab: unstructured (26)            |-62.62 (10.73) |          -5.83|       -1.65|<.0001           |
|Other: unstructured (128)         |-60.22 (3.23)  |         -18.63|       -2.34|<.0001           |
|Unreported (105)                  |-23.1 (3.33)   |          -6.93|       -0.96|<.0001           |


By context, dropping English

In [31]:
# drop English
rand_dat_inc_cg_no_eng = rand_dat_inc_cg[rand_dat_inc_cg["Language_name"] != "English"]

rand_lex_sumstats_contex_no_eng = (rand_dat_inc_cg_no_eng.groupby(["context","target_child_id","transcript_id","contingent"])
                                  .uniquenss
                                  .agg(["sum"])
                                  .reset_index())

rand_lex_sumstats_contex_no_eng =  rand_lex_sumstats_contex_no_eng.rename({'sum': 'sums'}, axis=1)

In [44]:
%%R -i rand_lex_sumstats_contex_no_eng

# # vectors for rows to remove from lmer
single_tran <- c("Home: interview/unstructured") # only 1 transcript

contex_sample_size_no_eng <- rand_lex_sumstats_contex_no_eng %>%
    group_by(context) %>%
    summarize(n = n_distinct(transcript_id))

lexdiv_contex_nest_1_no_eng <- rand_lex_sumstats_contex_no_eng %>%
    filter(!context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lmer(sums ~ contingent +
                                (1|transcript_id),
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise"))),
           effect_size = map2(summary, fit, ~ eff_size(.x, sigma = sigma(.y), edf = df.residual(.y)))) %>%
    select(context, contrasts, effect_size) %>%
    unnest(cols = c(contrasts)) %>%
    mutate(effect_size = map(effect_size, ~ summary(.))) %>%
    unnest() %>%
    mutate(statistic = coalesce(`t.ratio`), .before = p.value) %>%
    select (-c(`t.ratio`))

lexdiv_contex_nest_2_no_eng <- rand_lex_sumstats_contex_no_eng %>%
    filter(context %in% single_tran) %>%
    group_by(context) %>%
    nest() %>%
    mutate(fit = map(data, ~ lm(sums ~ contingent,
                                data = .,
                                REML= FALSE)),
           summary = map(fit, ~ emmeans(., "contingent")),
           contrasts = map(summary, ~ summary(contrast(., method = "pairwise")))) %>%
    select(context, contrasts) %>%
    unnest(cols = c(contrasts))  %>%
    rename(statistic = `t.ratio`)
    
# combine lmer summaries and correct p-values for multiple comparisons
context_no_eng_emms_all <- list(lexdiv_contex_nest_1_no_eng, lexdiv_contex_nest_2_no_eng) %>% 
    reduce(bind_rows) %>%
    mutate(p.value = p.adjust(p.value, "holm", 3)) %>%
    left_join(contex_sample_size_no_eng)

R[write to console]: boundary (singular) fit: see help('isSingular')

R[write to console]: boundary (singular) fit: see help('isSingular')



Joining with `by = join_by(context)`


In [45]:
%%R 

table_maker = function(data) { data %>%
    select(context, n, estimate, SE, statistic, effect.size, p.value) %>%
    `colnames<-`(c("Play context", "n", "Estimate", "SE", "Test statistic", "Effect size", "Adjusted p-value")) %>%
    mutate_at(vars(-c(`Adjusted p-value`,`Play context`)), round,2) %>%
    mutate(`Adjusted p-value` = format(round(`Adjusted p-value`,4),nsmall=4)) %>%
    mutate(`Adjusted p-value` = gsub("0.0000","<.0001",`Adjusted p-value`)) %>%
    unite("Estimate (SE)", c('Estimate','SE'), sep=" (") %>%
    mutate(`Estimate (SE)` = paste0(`Estimate (SE)`,")")) %>%
    unite("Play context (n)", c(`Play context`,'n'), sep=" (") %>%
    mutate(`Play context (n)` = paste0(`Play context (n)`,")")) %>%
    arrange(`Play context (n)`)
    }
    
lexdiv_context_stats_table_no_eng <- table_maker(context_no_eng_emms_all)

kable(lexdiv_context_stats_table_no_eng)



|Play context (n)                 |Estimate (SE)   | Test statistic| Effect size|Adjusted p-value |
|:--------------------------------|:---------------|--------------:|-----------:|:----------------|
|Home: book reading (28)          |-116.71 (10.89) |         -10.72|       -2.92|<.0001           |
|Home: interview/unstructured (1) |-13 (NaN)       |            NaN|          NA|NaN              |
|Home: unstructured (560)         |-64.37 (2.12)   |         -30.41|       -1.83|<.0001           |
